`Xarray-DataAccessor` + `ModelMyWatershed API` + `FCPGtools` Demo Notebook
==========================================================================

**Author:** [Xavier R Nogueira](https://github.com/xaviernogueira)

**Notebook Steps:**
1. Using the `xarray_data_accessor.DataAccessorFactory` library to search available datasets/variables.
2. Use the Model My Watershed API to delineate a the watershed boundary for our lat/lon (CONUS only).
3. Read-in hourly ERA5 precipitation data from [Planet OS's AWS S3 bucket](https://github.com/planet-os/notebooks/blob/master/aws/era5-pds.md) for our South American basin of choice using `xarray_data_accessor`.
4. Read in [NASA Digital Elevation Model (DEM) data](https://lpdaac.usgs.gov/products/nasadem_hgtv001/) for out watershed using `xarray_data_accessor`.
5. Convert our DEM to a D8 Flow Direction Raster using `pysheds`.
6. Use `fcpgtools` to resample our precipitation data to match the resolution of the FDR.
7. Use `fcpgtools` to calculate precipitation accumulation for all hourly time steps.

All the while we will be creating interactive plots/maps using [`hvplot`](https://hvplot.holoviz.org/) and [`geoviews`](https://geoviews.org/).

**OR just get the aspect directly** https://lpdaac.usgs.gov/products/nasadem_scv001/

Investigate reading with fsspec? https://github.com/microsoft/AIforEarthDataSets/blob/main/data/nasadem-nc.ipynb

In [ ]:
# import dependencies
import requests
import geopandas
import shapely
import xarray as xr
import numpy as np
import hvplot.xarray
import hvplot.pandas
import cartopy.crs as ccrs
from pathlib import Path
import gc
import time
import json
import pysheds
from typing import (
    List,
    Optional,
    TypedDict,
)

# import our library
import xarray_data_accessor

# import our username/password
import auth_config
from auth_config import (
    MMW_AUTH_DICT,
    EARTH_DATA_AUTH_DICT,
)

In [ ]:
# env not liking fcpgtools - I cloned the repo and added it to my environment (also pip installed pysheds)
import fcpgtools

# 🗃️ Explore data availability 🗃️

## First we see what datasets can be accessed by different 'data accessors"

In [ ]:
# lets start by seeing which DataAccessor objects are currently available
xarray_data_accessor.DataAccessorFactory.data_accessor_names()

In [ ]:
# next lets see what datasets each can access
xarray_data_accessor.DataAccessorFactory.supported_datasets()

## Next we see which ERA5 hourly variables can be fetched with the `AWSDataAccessor`

In [ ]:
xarray_data_accessor.DataAccessorFactory.supported_variables(
    data_accessor_name='AWSDataAccessor',
    dataset_name='reanalysis-era5-single-levels',
)

## We also explore NASA elevation products

In [ ]:
xarray_data_accessor.DataAccessorFactory.supported_variables(
    data_accessor_name='NASA_LPDAAC_Accessor',
    dataset_name='NASADEM_NC',
)

# 🌄 Define a watershed AOI using Model My Watershed API 🌄

In [ ]:
from shapely.geometry import (
    Polygon,
    Point,
)

In [ ]:
# select a lat/long in CONUS
lat = 38.971125
lon = -77.042225

# get point as a GeoDataFrame
point_gdf = geopandas.GeoDataFrame({'geometry': [Point((lon, lat))]})

In [ ]:
mmw_base_url = r'https://modelmywatershed.org/api'

In [ ]:
%%time
# get a token code
token_response = requests.post(
    mmw_base_url + '/token/',
    data=auth_config.MMW_AUTH_DICT,
)
token = dict(token_response.json())['token']

## Define function to handle API request

In [ ]:
class LocationDict(TypedDict):
    location: List[float]
    snappingOn: Optional[str]
    simplify: Optional[int]
    datasource: Optional[str]


def get_watershed(
    session: requests.Session,
    location_dict: LocationDict,
    mmw_base_url: str,
) -> geopandas.GeoDataFrame:
    """Uses the Model My Watershed API to get a boundary"""
    # init the job
    job_dict = mmw_session.post(
        mmw_base_url + '/watershed/',
        data=json.dumps(location_dict),
    )

    # use the job ID to check up on it
    done = False
    while not done:
        status_dict = dict(session.get(mmw_base_url + f'/jobs/{dict(job_dict.json())["job"]}').json())
        if status_dict['status'] == 'complete':
            done = True
            print('Delineation worked! Returning results...')
        elif status_dict['status'] == 'failed':
            raise ValueError(f'API request failed! See response: {status_dict}')

    watershed_dict = status_dict['result']['watershed']
    watershed_poly = Polygon(watershed_dict['geometry']['coordinates'][0])
    return geopandas.GeoDataFrame({'geometry': [watershed_poly]}, crs='EPSG:4326')

In [ ]:
# start a requests session
mmw_session = requests.Session()
mmw_session.headers.update({
    'Authorization': 'Token ' + token,
    'Content-Type': 'application/json'
})

In [ ]:
location_dict = {
    'location': [lat, lon],
    'snappingOn': True,
    'dataSource': 'nhd',
}

In [ ]:
%%time
watershed = get_watershed(
    mmw_session,
    location_dict,
    mmw_base_url,
)

In [ ]:
# plot our basin(s) and point
watershed.hvplot(
    crs=watershed.crs.to_wkt(),
    tiles='StamenTerrainRetina',
    width=500,
    height=500,
    fill_color=None,
    line_width=4,
    line_color='blue',
) * point_gdf.hvplot(
    color='red',
    crs=watershed.crs.to_wkt(),
)

# 🛰️ Read-in ERA5 precipitation data from the Planet OS AWS cloud bucket 🛰️

In [ ]:
%%time
xarray_data = xarray_data_accessor.get_xarray_dataset(
    data_accessor_name='AWSDataAccessor',
    dataset_name='reanalysis-era5-single-levels',
    variables=[
        'precipitation_amount_1hour_Accumulation',
    ],
    start_time='2021-08-09',
    end_time='2021-08-12',
    shapefile=watershed,
)

In [ ]:
xarray_data

In [ ]:
xarray_data.precipitation_amount_1hour_Accumulation.hvplot(
    x='time',
)

# 🛰️ Read-in NASA DEM data from their API 🛰️

In [ ]:
%%time
# TODO: CHANGE THIS
dem_data = xarray_data_accessor.get_xarray_dataset(
    data_accessor_name='NASA_LPDAAC_Accessor',
    dataset_name='NASADEM_NC',
    variables=[
        'DEM',
    ],
    start_time='2021-07-15',
    end_time='2021-07-18',
    shapefile=watershed,
)

In [ ]:
dem_data

In [ ]:
# plot a central point over time
gc.collect()
dem_data.NASADEM_HGT.hvplot(
    crs=watershed.crs.to_wkt(),
    tiles='StamenTerrainRetina',
    width=500,
    height=500,
) * watershed.hvplot(
    crs=watershed.crs.to_wkt(),
    fill_color=None,
    line_width=4,
    line_color='green',
) * point_gdf.hvplot(
    color='red',
    crs=watershed.crs.to_wkt(),
)

# 🧰 Prep `FCPGtools` Inputs 🧰

## Convert DEM to a D8 Flow Direction Raster

In [ ]:
%%time
# convert to slope
slope = np.arctan(dem_data.NASADEM_HGT.differentiate('lat') / np.sqrt(dem_data.NASADEM_HGT.differentiate('lon')**2 + dem_data.NASADEM_HGT.differentiate('lat')**2))

# fill flats and nodata
slope = slope.interpolate_na(dim='lat', method='linear')
del dem_data

In [ ]:
# convert to aspect
aspect = np.rad2deg(np.arctan2(slope.differentiate('lon'), slope.differentiate('lat')))
del slope

In [ ]:
def aspect_to_d8(aspect_angle: int, d8_format: str = 'esri') -> xr.DataArray:
    d8_direction = np.full(aspect_angle.shape, 255)

    # get the D8 FDR values from fcpgtools
    conv_dict = fcpgtools.custom_types.D8ConversionDicts[d8_format]

    d8_direction[(aspect_angle >= 337.5) | (aspect_angle < 22.5)] = conv_dict['north']
    d8_direction[(aspect_angle >= 22.5) & (aspect_angle < 67.5)] = conv_dict['northeast']
    d8_direction[(aspect_angle >= 67.5) & (aspect_angle < 112.5)] = conv_dict['east']
    d8_direction[(aspect_angle >= 112.5) & (aspect_angle < 157.5)] = conv_dict['southeast']
    d8_direction[(aspect_angle >= 157.5) & (aspect_angle < 202.5)] = conv_dict['south']
    d8_direction[(aspect_angle >= 202.5) & (aspect_angle < 247.5)] = conv_dict['southwest']
    d8_direction[(aspect_angle >= 247.5) & (aspect_angle < 292.5)] = conv_dict['west']
    d8_direction[(aspect_angle >= 292.5) & (aspect_angle < 337.5)] = conv_dict['northwest']

    return d8_direction

In [ ]:
%%time
fdr = xr.apply_ufunc(
    aspect_to_d8,
    aspect.compute(),
    output_dtypes=[np.int32],
)
fdr = fdr.rio.write_crs(4326)
del aspect
gc.collect()

## Clip to our FDR to our basin AOI

In [ ]:
# clip to the bbox
fdr = fcpgtools.clip(
    fdr,
    match_shapefile=watershed,
)
# mask to only include values in our shapefile
fdr = fcpgtools.spatial_mask(
    fdr,
    mask_shp=watershed,
)

In [ ]:
fdr

In [ ]:
fdr.where(fdr != 255, np.nan).hvplot(
    crs=fdr.rio.crs.to_wkt(),
    tiles='StamenTerrainRetina',
    width=500,
    height=500,
    clim=(0, 250),
    cmap='Category10',
) * watershed.hvplot(
    crs=watershed.crs.to_wkt(),
    fill_color=None,
    line_width=4,
    line_color='black',
) * point_gdf.hvplot(
    color='red',
    crs=watershed.crs.to_wkt(),
)

## Align our ERA5 data with the prepped FDR

In [ ]:
%%time
aligned_era5 = fcpgtools.align_raster(
    xarray_data['precipitation_amount_1hour_Accumulation'],
    fdr,
    resample_method='bilinear',
)

In [ ]:
aligned_era5

In [ ]:
# plot our basin(s) and point
aligned_era5.hvplot.image(
    crs=aligned_era5.rio.crs.to_wkt(),
    clim=(
        aligned_era5.min().item() + 0.00001,
        aligned_era5.max().item()
    ),
    cmap='PuBu',
    cnorm='log',
    width=600,
    height=500,
    widget_type='scrubber',
    widget_location='bottom',
    tiles='StamenTerrainRetina',
)

# 🌧️ Calculate flow accumulation over time 🌧️

In [ ]:
%%time
gc.collect()
flow_accum = fcpgtools.accumulate_parameter(
    fdr,
    aligned_era5,
    engine='pysheds',
)

In [ ]:
flow_accum

In [ ]:
# sanity check that accumulation matches
flow_accum[:, 400, 400].hvplot(
    x='time',
)

In [ ]:
# plot our basin(s) and point
flow_accum.hvplot.image(
    crs=flow_accum.rio.crs.to_wkt(),
    clim=(
        flow_accum.min().item() + 0.00001,
        flow_accum.max().item()
    ),
    cmap='PuBu',
    cnorm='log',
    width=600,
    height=500,
    widget_type='scrubber',
    widget_location='bottom',
    tiles='StamenTerrainRetina',
)